In [1]:
model_name = "ai4bharat/indic-bert"

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset, Dataset, ClassLabel
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
import pandas as pd

torch.cuda.empty_cache()

In [3]:
df = pd.read_csv('Balanced.csv')

In [4]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)


In [5]:
data_train = Dataset.from_pandas(train_data, preserve_index=False)
data_test = Dataset.from_pandas(test_data, preserve_index=False)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

labels = ClassLabel(num_classes=4, names=['Offensive-Ind', 'Not-Offensive', 'Offensive-Group','Offensive-Untargetted'], names_file=None, id=None)

def tokenize_function(examples):
    tokens =  tokenizer(examples["text"], padding=True, truncation=True, max_length=128)
    tokens['label'] = labels.str2int(examples['label'])
    return tokens

tokenized_train = data_train.map(tokenize_function, batched=True)
tokenized_test = data_test.map(tokenize_function, batched=True)

tokenized_train.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_test.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Parameter 'function'=<function tokenize_function at 0x7f112870f1f0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
id2label = {idx:labels.int2str(idx) for idx in range(4)}
label2id = {v:k for k,v in id2label.items()}

In [9]:
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        #Feed inputs to model and extract logits
        outputs = model(**inputs)
        logits = outputs.get("logits")
        #Extract labels
        labels = inputs.get("labels")
        # Define loss function with class weights
        loss_func = nn.CrossEntropyLoss()
        # Compute loss
        loss = loss_func(logits, labels)
        return (loss, outputs) if return_outputs else loss 

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                             num_labels=4,
                                                             id2label=id2label,
                                                             label2id=label2id)

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'sop_classifier.classifier.bias', 'sop_classifier.classifier.weight', 'predictions.dense.bias', 'predictions.dense.weight', 'predictions.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

In [11]:
def compute_metrics(pred):

    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    accuracy = accuracy_score(y_true=labels, y_pred=preds)
    recall = recall_score(y_true=labels, y_pred=preds, average='weighted')
    precision = precision_score(y_true=labels, y_pred=preds, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=preds, average='weighted')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [12]:
batch_size = 16
epochs = 30
leaning_rate = 2e-5


# Log the training loss at each epoch
logging_steps = len(tokenized_train) // batch_size

training_args = TrainingArguments(output_dir='Indic-BERT',
                                    num_train_epochs=epochs,
                                    learning_rate=leaning_rate,
                                    per_device_train_batch_size=batch_size,
                                    per_device_eval_batch_size=batch_size,
                                    weight_decay=0.01,
                                    evaluation_strategy="steps",
                                    logging_steps=logging_steps,
                                    fp16=True,
                                    eval_steps=250,
                                    load_best_model_at_end=True)

In [13]:
trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

Using cuda_amp half precision backend


In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 800
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 750


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
250,0.841500,1.369944,0.480000,0.230400,0.480000,0.311351
500,0.540700,1.483090,0.550000,0.470033,0.550000,0.494082
750,0.229300,1.715691,0.510000,0.536182,0.510000,0.502164


The following columns in the evaluation set don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 32
Saving model checkpoint to Indic-BERT/checkpoint-500
Configuration saved in Indic-BERT/checkpoint-500/config.json
Model weights saved in Indic-BERT/checkpoint-500/pytorch_model.bin
tokenizer config file saved in Indic-BERT/checkpoint-500/tokenizer_config.json
Special tokens file saved in Indic-BERT/checkpoint-500/special_tokens_map.js

TrainOutput(global_step=750, training_loss=0.6809565823872884, metrics={'train_runtime': 103.8509, 'train_samples_per_second': 231.101, 'train_steps_per_second': 7.222, 'total_flos': 143416958976000.0, 'train_loss': 0.6809565823872884, 'epoch': 30.0})

In [1]:


# from transformers import TextClassificationPipeline, AutoModelForSequenceClassification, AutoTokenizer
# from sklearn.metrics import classification_report
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from datasets import  ClassLabel


# df = pd.read_csv('Balanced.csv')
# train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

model_name = "Indic-BERT/checkpoint-500"
tokenizer = AutoTokenizer.from_pretrained(model_name)



labels = ClassLabel(num_classes=4, names=['Offensive-Ind', 'Not-Offensive', 'Offensive-Group','Offensive-Untargetted'], names_file=None, id=None)
id2label = {idx:labels.int2str(idx) for idx in range(4)}
label2id = {v:k for k,v in id2label.items()}

model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=4,id2label=id2label,label2id=label2id)


pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=False)
y_pred = pipe(test_data['text'].tolist())
y_pred = [y_pred[i]["label"] for i in range(len(y_pred))]
y_true = test_data['label'].tolist()
print(classification_report(y_true, y_pred))

/home/sudheesh/anaconda3/envs/rubrix/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:89: UserWarning: `return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality
  warnings.warn(


                       precision    recall  f1-score   support

        Not-Offensive       0.62      0.88      0.72        96
      Offensive-Group       0.00      0.00      0.00        27
        Offensive-Ind       0.27      0.28      0.28        39
Offensive-Untargetted       0.61      0.37      0.46        38

             accuracy                           0.55       200
            macro avg       0.37      0.38      0.36       200
         weighted avg       0.46      0.55      0.49       200



/home/sudheesh/anaconda3/envs/rubrix/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sudheesh/anaconda3/envs/rubrix/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sudheesh/anaconda3/envs/rubrix/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le